In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

import time
import os

######################## define constants etc. ############################### 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#rootpath = '/n/holyscratch01/dvorkin_lab/Users/tianliwang/maps_unzipped/'  # location of maps 
rootpath = '/Volumes/WTL/convergence_maps/All_cosm/'  # <-- Tianli's testing file path
workPath = '/Users/Ed/Desktop/cosm_prjt/'  # path to load the net from and write the predictions in

# these parameters are defined in the paper  
n_imagespercosmo = 512  # number of images per cosmology 
n_perbatch = 32         # batch size for training 
n_perbatch_test = n_perbatch  # the value shoudn't matter but put here in case we need to change things
n_batch = n_imagespercosmo/n_perbatch  # number of minibatches 
dim_image = 1024 
dim_downsized = int(dim_image/2)            # downsized image dimension 

trainFraction = 14.0/16  # fraction of images per cosmology for training 
validateFraction = 0.0/16
testFraction = 1-trainFraction-validateFraction  # needs more cleaning up 

n_cosmosToTrain = 2  # same for validation and testing
n_trainimages = int(n_imagespercosmo*trainFraction)    # number of images per cosmology to train
n_validateimages = int(n_imagespercosmo*validateFraction)   # number of images per cosmology to validate
n_testimages = int(n_imagespercosmo*testFraction)  # number of images per cosmology to test
n_batch_train = int(n_trainimages/n_perbatch)  # number of minibatches per cosmology to train 

# start and end image indices for training and validating 
startIndices_train = np.ones(n_cosmosToTrain).astype(int)
endIndices_train = n_trainimages*np.ones(n_cosmosToTrain).astype(int)
startIndices_validate = (n_trainimages+1)*np.ones(n_cosmosToTrain).astype(int)
endIndices_validate = (n_trainimages+n_validateimages)*np.ones(n_cosmosToTrain).astype(int)
startIndices_test = (n_trainimages+n_validateimages+1)*np.ones(n_cosmosToTrain).astype(int)
endIndices_test = (n_trainimages+n_validateimages+n_testimages)*np.ones(n_cosmosToTrain).astype(int)

print(startIndices_test, endIndices_test)  # shoud be 481 and 512 (just testing)


######################## define functions & Class ###############################

def read_parametersfromfile(path):
    '''
    Input: string for the directory to the folders of cosmology 
    
    Function: reads the Omega_m and sigma_8 values from the folder names in 
    the path directory and store them as arrays of strings. Stores the values 
    as strings to avoid rounding, for example, 0.600 to 0.6. 
    
    Output: (array of strings of Omega_m, array of strings of sigma_8)
    '''
    
    Om_strings = [] 
    si_strings = []
    
    for filename in os.listdir(path):
        if (filename.startswith('Om') and not filename.endswith('.gz')): 
            Om_strings.append(filename[2:7]) 
            si_strings.append(filename[10:15])
            
    return Om_strings, si_strings


def read_files(path, Oms, sis, start_indices, end_indices): 
    '''
    Input: file root directory path, strings of omegam and sigma8 values, 
    start image indices and end image indices. Oms, sis, start_indices, end_indices
    should be arrays of same length. 
    
    Function: reads the images labeled between start_indices[i] and end_indices[i] (inclusive)
    for each of the ith cosmologies specified by omegam and sigma8 values 
    
    Output: numpy array of 3D images
    '''
    
    n_cosmos = len(Oms)  # this should be shared by Oms, sis, start_indices, end_indices 
    
    # read in the files and put them into 3D matrix 
    filedata = []

    for j in range(0, n_cosmos): 
        i_start = start_indices[j]
        i_end = end_indices[j]
        Om = Oms[j]
        si = sis[j]
        
        for i in range(i_start, i_end+1): 
            if (i < 10): filenum = '00{}'.format(i)
            elif (i < 100): filenum = '0{}'.format(i)
            else: filenum = i 

            hdulist = fits.open('{}Om{}_si{}/WLconv_z1.00_0{}r.fits'.format(path, Om, si, filenum))
            image = hdulist[0].data

            # downsize from dim_image to dim_downsized 
            image_downsized = image.reshape([dim_downsized, dim_image//dim_downsized, 
                                    dim_downsized, dim_image//dim_downsized]).mean(3).mean(1)

            # newaxis makes the image 3D (1x512x512) 
            filedata.append(image_downsized[np.newaxis,:,:])
    
    return np.array(filedata)
           
    
def test_cnn(network, testinputs, batchsize, isTrackingloss, **kwargs): 
    '''
    Input: CNN object, testing data, batch size for testing, boolean for whether 
    to track losses bewteen test ouputs and targets, **kwargs includes the loss function
    criterion and the targetting outputs. Target should be in the form of 
    an array of (omegam, sigma8) parameters each corresponding to the target of one batch. 
    
    Call with 
    test_cnn(network, testinputs, batchsize, False) 
    or 
    test_cnn(network, testinputs, batchsize, True, loss_fn=loss_fn, target=target)
    
    Function: Run the network with test inputs and specified batch size and track loss as 
    requested. 
    
    Output: 
    If isTrackingloss is True, return (np array of predicted outputs, np array of losses for each batch)
    If isTrackingloss is False, return np array of predicted outputs
    '''
    
    # unpack optional arguments **kwargs 
    if (isTrackingloss): 
        loss_fn = kwargs.get('loss_fn', None)
        testtargets = kwargs.get('target', None)
        losses = []
    
    # make the data into 4D tensor and put each batch into iterable 
    testloader = torch.utils.data.DataLoader(testinputs, batch_size=batchsize, shuffle=False)
    
    predictions = []  # stores outputs of network with test data inputs 
    
    with torch.no_grad(): 
        for i, testdata in enumerate(testloader, 0):
            testdata = testdata.to(device)  # GPU
            testoutputs = network(testdata)
            if i==0:
                predictions = (testoutputs.cpu()).numpy()
            if i!=0:
                predictions = np.concatenate((predictions, (testoutputs.cpu()).numpy()),0)
#            predictions.append((testoutputs.cpu()).numpy())  # put tensor back into cpu before converting to np 
##            print(predictions)

            if (isTrackingloss): 
                testtarget = torch.tensor(testtargets[i]).repeat(batchsize, 1)  
##                print(testtarget)
                testtarget = testtarget.to(device)  # GPU
                loss = np.array([(loss_fn(testoutputs[i], testtarget[i])).cpu() for i in range(batchsize)])
                losses.append(loss)
##                print(loss)
    
    if (isTrackingloss): 
        return np.array(predictions), np.array(losses)
    
    return np.array(predictions)


cpu
[449 449] [512 512]


In [2]:
# network structure 

def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class Net (nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            # layers 1+2 
            conv_block(1, 32, kernel_size=3), 
            conv_block(32, 32, kernel_size=3), 
            nn.AvgPool2d(2, 2), 
            # layers 3+4
            conv_block(32, 64, kernel_size=3), 
            conv_block(64, 64, kernel_size=3), 
            nn.AvgPool2d(2, 2), 
            # layers 5-7 
            conv_block(64, 128, kernel_size=3),
            conv_block(128, 64, kernel_size=1),
            conv_block(64, 128, kernel_size=3),
            nn.AvgPool2d(2, 2),
            # layers 8-10 
            conv_block(128, 256, kernel_size=3),
            conv_block(256, 128, kernel_size=1),
            conv_block(128, 256, kernel_size=3),
            nn.AvgPool2d(2, 2),
            # layers 11-13 
            conv_block(256, 512, kernel_size=3),
            conv_block(512, 256, kernel_size=1),
            conv_block(256, 512, kernel_size=3),
            nn.AvgPool2d(2, 2),
            # layers 14-18 
            conv_block(512, 512, kernel_size=3),
            conv_block(512, 256, kernel_size=1),
            conv_block(256, 512, kernel_size=3),
            conv_block(512, 256, kernel_size=1),
            nn.Conv2d(256, 512, kernel_size=3),
            nn.ReLU(),             # no batch norm on last convolution layer 
            nn.AvgPool2d(6, 6)
        )
        
        self.fc = nn.Linear(512, 2)

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(-1, 512)
        x = self.fc(x)
        
        return x

In [3]:
# read in all the omegam and sigma8 values from the directory 
Omegam_strings, sigma8_strings = read_parametersfromfile(rootpath)

Omegastrings_test = Omegam_strings[:n_cosmosToTrain]
sigmastrings_test = sigma8_strings[:n_cosmosToTrain]
Omegas_test = np.array(Omegastrings_test).astype(np.float)
sigmas_test = np.array(sigmastrings_test).astype(np.float)

start_time = time.time()  
inputs_test = read_files(rootpath, Omegastrings_test, sigmastrings_test, 
                             startIndices_test, endIndices_test)
print("--- Time to load the test files: %s seconds ---" % (time.time() - start_time), flush=True)

--- Time to load the test files: 21.55128002166748 seconds ---


In [5]:
net = Net()
net.to(device)  #GPU
path_net = workPath+'fullNet_gpu_90Cosmo_30epoch.pth'
net.load_state_dict(torch.load(path_net, map_location=torch.device('cpu')))  # delet the cpu option

--- Time to load the network: 0.10053396224975586 seconds ---


In [6]:
# Testing

start_time = time.time() 

n_batch_test = int(n_testimages/n_perbatch_test)
targets_temp = np.concatenate((np.array([Omegas_test]).T, np.array([sigmas_test]).T), axis=1) # temp stacked targets of all comsos being trained
targets_test = np.repeat(targets_temp, n_batch_test, axis=0)  # repeat each set of parameters by the number of batches
#outputs_test = test_cnn(net, inputs_test, n_perbatch_test, False)  # not tracking loss
criterion = nn.L1Loss()  # MAE loss 
outputs_test, losses_test = test_cnn(net, inputs_test, n_perbatch_test, 
                                                 True, loss_fn=criterion, target=targets_test)    
    

## print('Epoch %d test loss: %.3f' % (epoch + 1, np.sum(losses_test)/len(losses_test)), flush=True)
## losseslist_test.append(np.sum(losses_validate)/len(losses_validate))

print("--- Testing time: %s seconds ---" % (time.time() - start_time), flush=True)

--- Testing time: 108.16155290603638 seconds ---


In [7]:
t = Table([outputs_test[:,0], outputs_test[:,1], np.ndarray.flatten(losses_test)], names=('Om_pred', 'si_pred', 'Losses'))
t.write(workPath+'fullNet_testing.dat', format='ascii', overwrite=True)



In [8]:
Om_predict = outputs_test[:,0]
si_predict = outputs_test[:,1]
print(Om_predict)
print(si_predict)

[0.2402521  0.24192996 0.2640354  0.23442587 0.24141914 0.25778732
 0.24404906 0.23587981 0.24339312 0.23608756 0.22818284 0.2669407
 0.24347457 0.26057404 0.2695773  0.26412612 0.23725295 0.23034894
 0.2562923  0.23679775 0.2314605  0.25010455 0.25244623 0.22545518
 0.24118912 0.26298046 0.24508491 0.25230333 0.23755784 0.2330363
 0.24608228 0.23817505 0.24577206 0.24046235 0.235742   0.24094792
 0.25538236 0.24315317 0.2245759  0.25516966 0.25729108 0.23314254
 0.25140643 0.23515302 0.23051041 0.2622915  0.2370539  0.23921925
 0.24312192 0.23759134 0.2435503  0.22087522 0.2378199  0.2497895
 0.23029692 0.24009512 0.24186599 0.24455848 0.24798293 0.24157794
 0.25575858 0.23048978 0.26009265 0.25863948 0.25188035 0.23765394
 0.2639043  0.2534419  0.2588077  0.25156787 0.24000975 0.24882916
 0.25539374 0.2509758  0.2457375  0.26622602 0.24732047 0.26414075
 0.26247352 0.27066088 0.25355196 0.24339665 0.2542204  0.2400224
 0.23716442 0.259849   0.24480706 0.23798664 0.24849162 0.26846826